In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import plotly.graph_objects as go
import networkx as nx

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')
from keras.preprocessing.text import Tokenizer
from gensim.models import KeyedVectors
from keras.layers import Embedding
from keras.models import Sequential
import tensorflow as tf

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import Model
import cv2
import os

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from spektral.data.loaders import SingleLoader, Loader
from spektral.datasets.citation import Citation
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import LayerPreprocess

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gsevr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('dev_sent_emo.csv',encoding='utf-8')
df['Utterance'] = df.Utterance.str.replace('',"'")
df['gender'] = df.Speaker.apply(lambda x: 'male' if x == 'Ross' or x == 'Joey' or x == 'Chandler' else 'female')
df = df.sample(frac=0.03,ignore_index=True)
df

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,gender
0,99,"Someone I can learn from, someone-someone who'...",Joey,neutral,neutral,9,6,6,3,"00:05:43,676","00:05:47,304",male
1,473,Sure! That would be nice.,Jen,joy,positive,44,5,5,17,"00:13:45,366","00:13:47,116",female
2,585,In a minute!!,Joey,anger,negative,56,6,3,22,"00:20:05,454","00:20:07,455",male
3,720,It-it would really help me out if you guys wer...,Ross,sadness,negative,68,8,6,21,"00:02:40,910","00:02:45,831",male
4,1148,"Uh, it's from yore. Like the days of yore. Y'k...",Rachel,neutral,neutral,110,3,6,11,"00:06:59,335","00:07:03,380",female
5,1152,I bet it has a great story behind it too.,Phoebe,joy,positive,110,6,6,11,"00:07:09,387","00:07:11,596",female
6,481,What?,Monica,neutral,neutral,45,1,3,17,"00:08:53,658","00:08:55,283",female
7,898,"I'm not saying it has to be right now, but I'm...",Chandler,neutral,neutral,83,5,8,23,"00:10:48,094","00:10:53,307",male
8,484,Aren't you gonna go?,Phoebe,neutral,neutral,45,4,3,17,"0:09:04,961","0:09:05,733",female
9,35,"Uh, actually I find Marion's views far too pro...",Woman,neutral,neutral,4,7,7,7,"00:15:12,036","00:15:16,623",female


In [3]:
def preprocess(text):
    t = text.lower()
    t = re.sub('\d+',r'',t)
    t = re.sub(r'\W+',r' ',t)
    return t

lemmatizer = WordNetLemmatizer()



df['prepro'] = [' '.join([lemmatizer.lemmatize(preprocess(txt))])
                 .strip() for txt in df['Utterance']]


texts = df.prepro.values
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=30, padding='post')

In [4]:
glove_path = 'glove.6B.50d.txt'
word_vectors = KeyedVectors.load_word2vec_format(glove_path, binary=False, encoding='utf8',no_header=True)



In [6]:
embedding_dim = 50  
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))

for word, i in tokenizer.word_index.items():
    if word in word_vectors:
        embedding_matrix[i] = word_vectors[word]

embedding_layer = Embedding(
    input_dim=len(tokenizer.word_index) + 1,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    input_length=30,
    trainable=False
)


model = Sequential()
model.add(embedding_layer)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics='accuracy')


In [7]:
embedding_vectors = model.predict(padded_sequences)
embedding_vectors.shape

2/2 [==============================] - 0s 2ms/step


(33, 30, 50)

In [8]:
df['text'] = list(embedding_vectors)


In [9]:
base_model = InceptionV3(weights='imagenet', include_top=False)

model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)

def get_video_embeddings(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_embeddings = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (299, 299))
        x = np.expand_dims(frame, axis=0)

        x = preprocess_input(x)

        frame_embedding = model.predict(x)

        frame_embeddings.append(frame_embedding)

    cap.release()
    return frame_embeddings



video_embeddings_list = []
folder_path = 'dev_splits_complete/'

for idx,row in df.iterrows():
    file_name = 'dia' + str(row['Dialogue_ID']) + '_utt' + str(row['Utterance_ID']) + '.mp4'
    video_path = folder_path + file_name
    if os.path.isfile(video_path):
        video_embeddings_list.append(get_video_embeddings(video_path))
    else:
        print(f'File name {file_name} does not exist')

df['video'] = video_embeddings_list
        


1/1 [==============================] - 0s 239ms/step


In [10]:
G = nx.Graph()

emotion_col = {
            'sadness':'teal',
            'surprise':'silver',
            'neutral':'lightgreen',
            'joy':'purple',
            'anger':'black',
            'disgust': 'brown',
            'fear':'yellow'
}

for emotion in df.Emotion.unique():
    G.add_node(emotion, color=emotion_col[emotion],hover=emotion,emotion=emotion, type='emotion')


In [11]:
gender_col = {
    'male':'blue',
    'female':'pink'
}

for gender in df.gender.unique():
    G.add_node(gender,color=gender_col[gender], hover=gender, gender=gender, type='gender')

In [13]:
for idx, row in df.iterrows():
    G.add_node(row['Utterance'], 
    text_embedding=row['text'],
    visual_embedding=row['video'],
    color=emotion_col[row['Emotion']],
    hover=row['Utterance'],
    emotion=row['Emotion'],
    gender=row['gender'], 
    type='utterance')

In [14]:
for idx, row in df.iterrows():
    G.add_edge(row['Utterance'], row['Emotion'],color=emotion_col[row['Emotion']])
    G.add_edge(row['Utterance'], row['gender'], color=gender_col[row['gender']])


In [15]:
pos = nx.spring_layout(G)
# pos = nx.fruchterman_reingold_layout(G)


In [19]:
import textwrap

fig = go.Figure()

node_positions = {}

for node, position in pos.items():
    x, y = position  

    color = G.nodes[node]['color']  
    hover = G.nodes[node]['hover']
    hover = '<br>'.join(textwrap.wrap(hover,width=50))
    try:
        emotion = G.nodes[node]['emotion']
    except KeyError:
        try:
            gender = G.nodes[node]['gender']
        except KeyError:
            continue
    node_type = G.nodes[node]['type']

    if node_type == 'utterance':
        fig.add_trace(go.Scatter(x=[x], y=[y], marker=dict(size=10,color=color), hovertext=hover, name=emotion, hovertemplate=hover))
    else:
        fig.add_trace(go.Scatter(x=[x], y=[y], marker=dict(size=10,color=color), hovertext=hover, name=node_type, hovertemplate=hover))


for edge in G.edges():
    color = G.edges[edge]['color']
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    fig.add_trace(go.Scatter(x=[x0, x1], y=[y0, y1], mode='lines', line=dict(width=1,color=color), name='Edge'))

fig.update_layout(showlegend=False, title='Emotions Graph')

fig.update_layout(xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))

fig.show()